In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Helper functions

In [3]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

### Uploading Data

In [4]:
"""
from google.colab import files
#drive.mount('/content/drive')

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes'.format(
      name=fn, length=len(uploaded[fn])))
"""

!wget -cq https://download.pytorch.org/tutorial/data.zip

import zipfile
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

## Loading and pre-processing data files

In [5]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()


def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs


MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991
['je suis comme ma mere', 'i am like my mother']


### Encoder Model

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

### Decoder with Attention

In [7]:
class BahdanauAttention(nn.Module):
    def __init__(self, dim_attention, dim_keys, dim_query):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(dim_attention, dim_query)
        self.Ua = nn.Linear(dim_attention, dim_keys)
        self.Va = nn.Linear(dim_attention, 1)

    def forward(self, query, keys, values):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, values)

        return context, weights


class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout=0.1):
        super(AttentionDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size, hidden_size, hidden_size)
        self.gru = nn.GRU(2*hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward_step(self, input, hidden, encoder_outputs):
        embedded = self.dropout(self.embedding(input))

        context, attention_weights = self.attention(query=hidden.permute(1, 0, 2), keys=encoder_outputs, values=encoder_outputs)

        input_gru = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attention_weights

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden= encoder_hidden
        decoder_outputs = []
        attentions = []

        # Run the decoder for MAX_LENGTH number of steps
        for i in range(MAX_LENGTH):
            # One step of decoder
            decoder_output, decoder_hidden, attention_weight = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_outputs.append(decoder_output)
            attentions.append(attention_weight)

            if target_tensor is not None:
                # Teacher forcing: using target as next input
                decoder_input = target_tensor[:, i].unsqueeze(1)
            else:
                # Using decoder output in this step as input to next step
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions



### Seq2Seq Model

In [8]:
class seq2seqAttention(nn.Module):
    def __init__(self, encoder, decoder):
        super(seq2seqAttention, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_tensor, target_tensor):
        # Forward propagation through encoder and decoder (one batch, MAX_LENGTH time steps)
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, attentions = decoder(encoder_outputs, encoder_hidden, target_tensor)
        return decoder_outputs, decoder_hidden, attentions

### Training script

In [ ]:
"""
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Optimizers and loss function
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        # Train network for on epoch
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    total_loss = 0

    for data in dataloader:
        input_tensor, target_tensor = data

        # Reset optimizers
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Forward propagation through encoder and decoder (one batch, MAX_LENGTH time steps)
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        # Compute loss: NLL
        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        # Back propagate gradients
        loss.backward()

        # Update network parameters
        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)
"""

def train(train_dataloader, model, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    # Optimizers and loss function
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        # Train network for on epoch
        loss = train_epoch(train_dataloader, model, optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

def train_epoch(dataloader, model, optimizer, criterion):
    total_loss = 0

    for data in dataloader:
        input_tensor, target_tensor = data

        # Reset optimizers
        optimizer.zero_grad()

        # Forward propagation through encoder and decoder (one batch, MAX_LENGTH time steps)
        decoder_outputs, _, _ = model(input_tensor, target_tensor)

        # Compute loss: NLL
        y = target_tensor.view(-1)
        y_pred = decoder_outputs.view(-1, decoder_outputs.size(-1))
        loss = criterion(y_pred, y)
        # Back propagate gradients
        loss.backward()

        # Update network parameters
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

### Train the network

In [9]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttentionDecoderRNN(hidden_size, output_lang.n_words).to(device)
model = seq2seqAttention(encoder, decoder).to(device)

Reading lines...
Read 135842 sentence pairs
Trimmed to 11445 sentence pairs
Counting words...
Counted words:
fra 4601
eng 2991


In [ ]:
#train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)
train(train_dataloader, model, 1, print_every=5, plot_every=5)

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

encoder.eval()
decoder.eval()
evaluateRandomly(encoder, decoder)

### PyTorch lightning

In [10]:
!pip install lightning
import lightning as L

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 62.8 MB/s eta 0:00:00


In [28]:
class LitSeq2seqAttention(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        x, y = batch
        encoder_outputs, encoder_hidden = encoder(x)
        decoder_outputs, decoder_hidden, attentions = decoder(encoder_outputs, encoder_hidden)
        logits = decoder_outputs.view(-1, decoder_outputs.size(-1))
        loss_fn = nn.NLLLoss()
        loss = loss_fn(logits, y.view(-1))
        return loss
    """
    def backward(self, trainer, loss, optimizer, optimizer_idx):
        loss.backward()

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx):
        optimizer.step()
    """

In [30]:
# init the autoencoder
modelLit = LitSeq2seqAttention(encoder, decoder)
trainer = L.Trainer(limit_train_batches=100, max_epochs=10)
trainer.fit(model=modelLit, train_dataloaders=train_dataloader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name    | Type                | Params
------------------------------------------------
0 | encoder | EncoderRNN          | 688 K 
1 | decoder | AttentionDecoderRNN | 950 K 
------------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.552     Total estimated model params size (MB)
INFO:

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.


## With Keras model.fit()


In [ ]:
import os

# This guide can only be run with the torch backend.
os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import layers

In [ ]:
class Seq2seqAttentionKeras(keras.Model):
    def __init__(self, encoder, decoder):
        super(seq2seqAttentionKeras, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def call(self, input_tensor, training=True):
        # Forward propagation through encoder and decoder (one batch, MAX_LENGTH time steps)
        print(input_tensor.type)
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, attentions = decoder(encoder_outputs, encoder_hidden)
        return decoder_outputs, decoder_hidden, attentions

    def train_step(self, data):
        for step, (inputs, targets) in enumerate(data):
            # Forward pass
            #decoder_outputs, _, _ = self(inputs)
            encoder_outputs, encoder_hidden = encoder(inputs)
            decoder_outputs, decoder_hidden, attentions = decoder(encoder_outputs, encoder_hidden)
            logits = decoder_outputs.view(-1, decoder_outputs.size(-1))
            loss = loss_fn(logits, targets.view(-1))

            # Backward pass
            modelKeras.zero_grad()
            trainable_weights = [v for v in modelKeras.trainable_weights]

            # Backpropagate loss to compute gradients for the weights.
            loss.backward()
            gradients = [v.value.grad for v in trainable_weights]

            # Update weights
            with torch.no_grad():
                optimizer.apply(gradients, trainable_weights)

            """
            # Log every 100 batches.
            if step % 100 == 0:
                print(
                    f"Training loss (for 1 batch) at step {step}: {loss.detach().cpu().numpy():.4f}"
                )
                print(f"Seen so far: {(step + 1) * batch_size} samples")
            """

        """
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        # Call torch.nn.Module.zero_grad() to clear the leftover gradients
        # for the weights from the previous train step.
        self.zero_grad()

        # Forward pass
        # encoder_outputs, encoder_hidden = self.encoder(x)
        # decoder_outputs, decoder_hidden, attentions = self.decoder(encoder_outputs, encoder_hidden, y)
        decoder_outputs, decoder_hidden, attentions = self(x.to(torch.int64), training=True)
        y_pred = decoder_outputs.view(-1, decoder_outputs.size(-1))

        # Compute loss
        loss = self.compute_loss(y=y, y_pred=y_pred)

        # Call torch.Tensor.backward() on the loss to compute gradients
        # for the weights.
        loss.backward()

        trainable_weights = [v for v in self.trainable_weights]
        gradients = [v.value.grad for v in trainable_weights]

        # Update weights
        with torch.no_grad():
            self.optimizer.apply(gradients, trainable_weights)
        """
        """
        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

        return loss
        """


In [ ]:
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttentionDecoderRNN(hidden_size, output_lang.n_words).to(device)
modelKeras = seq2seqAttentionKeras(encoder, decoder).to(device)

loss_fn = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for step, (inputs, targets) in enumerate(train_dataloader):
        # Forward pass
        decoder_outputs, _, _ = modelKeras(inputs)
        logits = decoder_outputs.view(-1, decoder_outputs.size(-1))
        loss = loss_fn(logits, targets.view(-1))

        # Backward pass
        model.zero_grad()
        loss.backward()

        # Optimizer variable updates
        optimizer.step()

In [ ]:
input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttentionDecoderRNN(hidden_size, output_lang.n_words).to(device)
modelKeras = seq2seqAttentionKeras(encoder, decoder).to(device)

loss_fn = nn.NLLLoss() #keras.losses.CategoricalCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

for epoch in range(10):
    print(f"\nStart of epoch {epoch}")
    for step, (inputs, targets) in enumerate(train_dataloader):
        # Forward pass
        decoder_outputs, _, _ = modelKeras(inputs)
        logits = decoder_outputs.view(-1, decoder_outputs.size(-1))
        loss = loss_fn(logits, targets.view(-1))

        # Backward pass
        modelKeras.zero_grad()
        trainable_weights = [v for v in modelKeras.trainable_weights]

        # Call torch.Tensor.backward() on the loss to compute gradients
        # for the weights.
        loss.backward()
        gradients = [v.value.grad for v in trainable_weights]

        # Update weights
        with torch.no_grad():
            optimizer.apply(gradients, trainable_weights)

        # Log every 100 batches.
        if step % 100 == 0:
            print(
                f"Training loss (for 1 batch) at step {step}: {loss.detach().cpu().numpy():.4f}"
            )
            print(f"Seen so far: {(step + 1) * batch_size} samples")

In [ ]:
modelKeras = seq2seqAttentionKeras(encoder, decoder).to(device)
input, target = next(iter(train_dataloader))
modelKeras(input)

modelKeras.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss=nn.NLLLoss(),
              metrics=[keras.metrics.CategoricalAccuracy()])

<built-in method type of Tensor object at 0x7ff16bd29670>


In [ ]:
history = modelKeras.fit(
    train_dataloader,
    epochs=1,
    steps_per_epoch = 1,
    validation_data=train_dataloader,
    validation_steps = 1,
) #callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])

<built-in method type of Tensor object at 0x7ff16bd0e9d0>
<built-in method type of Tensor object at 0x7ff16bd510d0>


RuntimeError: Unable to automatically build the model. Please build it yourself before calling fit/evaluate/predict. A model is 'built' when its variables have been created and its `self.built` attribute is True. Usually, calling the model on a batch of data is the right way to build it.
Exception encountered:
'Exception encountered when calling seq2seqAttentionKeras.call().

[1mExpected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)[0m

Arguments received by seq2seqAttentionKeras.call():
  • input_tensor=torch.Tensor(shape=torch.Size([32, 10]), dtype=float32)
  • training=True'

### Tensorflow + Keras

In [ ]:
import numpy as np

import typing
from typing import Any, Tuple

import tensorflow as tf

In [ ]:
class EncoderRNN(tf.keras.layers.Layer):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()

        # Embedding layer converts tokens to vectors
        self.embedding = tf.keras.layers.Embedding(input_size, hidden_size, mask_zero=True)
        # RNN layer to process sentence sequentially
        self.gru = tf.keras.layers.Bidirectional(
            merge_mode='sum',
            layer = tf.keras.layers.GRU(hidden_size, return_sequences=True,
                                        recurrent_initializer='glorot_uniform'))
    def call(self, x):
        x = self.embedding(x)
        x = self.rnn(x)
        return x

class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, hidden_size, **kwargs):
        self.mha = tf.keras.layers.MultiHeadAttention(key_dim=hidden_size, num_heads=1, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, input, context):
        attn_output, attn_scores = self.mha(query=input, value=context, return_attention_scores=True)
        x = self.add([input, attn_output])
        x = self.layernorm(x)

        return x

class AttentionDecoderRNN(tf.keras.layers.Layer):
    def __init__(self, output_size, hidden_size):
        super(AttentionDecoderRNN, self).__init__()
        # Embedding layer converts tokens to vectors
        self.embedding = tf.keras.layers.Embedding(output_size, hidden_size, mask_zero=True)
        # RNN layer to process sentence sequentially
        self.rnn = tf.keras.layers.GRU(hidden_size, return_sequences=True, return_state=True,
                                   recurrent_initializer='glorot_uniform')
        # Decoder RNN output will be the query for the attention layer.
        self.attention = CrossAttention(hidden_size)
        # Output layer to produce logits
        self.output_layer = tf.keras.layers.Dense(output_size)

    def call(self, input, context, state=None, return_state=False):
        # Lookup the embeddings
        x = self.embedding(input)
        # Process the target sequence.
        x, state = self.rnn(x, initial_state=state)
        # Use the RNN output as the query for the attention over the context.
        x = self.attention(x, context)
        # Generate logit predictions for the next token.
        logits = self.output_layer(x)

        if return_state:
          return logits, state
        else:
          return logits

class Translator(tf.keras.Model):
      def __init__(self, input_size, hidden_size, output_size):
          super().__init__()
          # Build the encoder and decoder
          encoder = Encoder(input_size, hidden_size)
          decoder = Decoder(output_size, hidden_size)

          self.encoder = encoder
          self.decoder = decoder

      def call(self, inputs):
          def call(self, inputs):
          context, x = inputs
          context = self.encoder(context)
          logits = self.decoder(x, context)
          return logits

      @tf.function
      # Compile into a static graph for faster training
      def train_step(self, data):
          x, y = data

          # Forward pass and compute loss
          with tf.GradientTape() as tape:
              logits = self(x, training=True)
              loss_value = self.loss(y, logits)

          # Backprop and compute gradients
          grads = tape.gradient(loss_value, model.trainable_weights)
          # Update weights
          self.optimizer.apply(grads, model.trainable_weights)

          return loss_value

model = Translator(input_size, hidden_size, output_size);
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')
optimizer = keras.optimizers.Adam(learning_rate=1e-3)

model.compile(optimizer=optimizer, loss=loss_fn)

history = model.fit(
    train_ds.repeat(),
    epochs=100,
    steps_per_epoch = 100,
    validation_data=val_ds,
    validation_steps = 20,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=3)])